## Callback EARLY STOPPING

In [1]:
# Importamos librerías a utilizar
import tensorflow as tf
import numpy as np

2024-04-19 17:03:51.601766: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Definición y entrenamiento inicial del modelo

Definimos un modelo sencillo y generamos un set de datos totalmente aleatorio

In [2]:
model = tf.keras.models.Sequential([tf.keras.layers.Dense(10)])
model.compile(tf.keras.optimizers.SGD(), loss='mse')
history = model.fit(np.arange(100).reshape(5, 20), np.zeros(5), epochs=10, batch_size=1,verbose=1)

Epoch 1/10
5/5 [==============================] - 0s 3ms/step - loss: 1499375694373715968.0000
Epoch 2/10
5/5 [==============================] - 0s 2ms/step - loss: 13133864187736317042908788397441024.0000
Epoch 3/10
5/5 [==============================] - 0s 2ms/step - loss: inf
Epoch 4/10
5/5 [==============================] - 0s 2ms/step - loss: inf
Epoch 5/10
5/5 [==============================] - 0s 2ms/step - loss: nan
Epoch 6/10
5/5 [==============================] - 0s 2ms/step - loss: nan
Epoch 7/10
5/5 [==============================] - 0s 2ms/step - loss: nan
Epoch 8/10
5/5 [==============================] - 0s 2ms/step - loss: nan
Epoch 9/10
5/5 [==============================] - 0s 2ms/step - loss: nan
Epoch 10/10
5/5 [==============================] - 0s 1ms/step - loss: nan


Utilizando el *callback* EarlyStopping, podemos diseñar una red neuronal de forma que el entrenamiento se detenga cuando el rendimiento del modelo no mejore.

### Aplicación y uso del callback EarlyStopping

In [3]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model = tf.keras.models.Sequential([tf.keras.layers.Dense(10)])
model.compile(tf.keras.optimizers.SGD(), loss='mse')
history = model.fit(np.arange(100).reshape(5, 20), np.zeros(5), epochs=10, batch_size=1, callbacks=[callback],verbose=1)

Epoch 1/10
5/5 [==============================] - 0s 1ms/step - loss: 16824020406108160.0000
Epoch 2/10
5/5 [==============================] - 0s 1ms/step - loss: 43732267061180643375983497373024256.0000
Epoch 3/10
5/5 [==============================] - 0s 1ms/step - loss: inf
Epoch 4/10
5/5 [==============================] - 0s 2ms/step - loss: inf


Vemos como, en esta ocasión, el entrenamiento se detiene despues de 4 épocas (*patience=3*). El parámetro *patience* del callback EarlyStopping se usa para establecer la cantidad de épocas que esperará el entrenamiento antes de detenerse si no mejora el rendimiento del modelo.

In [4]:
len(history.history['loss']) # Número de Épocas

4

Podemos ver cómo EarlyStopping ayuda a detener el entrenamiento monitorizando el rendimiento del modelo.    
Ahora imaginemos un caso en el que no sabemos cuántas épocas son necesarias para obtener una buena precisión en unos datos concretos. Para ese tipo de caso, EarlyStopping nos da la ventaja de establecer un número grande de épocas y establecer el valor del parámetro paciencia (*patience*) como 5 o 10 para detener el entrenamiento mediante la monitorización del rendimiento.


Aunque es posible utilizar la pérdida y la precisión del entrenamiento, EarlyStopping tiene sentido si se dispone de datos de Validación que pueden ser evaluados durante el entrenamiento.    
En función del rendimiento de estos datos de validación, se detendrá el entrenamiento.

Sintaxis a utilizar:
```python
model.fit(train_X, train_y, validation_split=0.3,callbacks=EarlyStopping(monitor=’val_loss’), patience=3)
````

En el ejemplo anterior, si la pérdida de validación ('val_loss') no disminuye durante 3 épocas consecutivas, el entrenamiento se detendrá.

### Parámetros de EarlyStopping

```python
tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)
```
El bloque de código anterior es la sintaxis y los parámetros, junto con los valores por defecto disponibles, para el callback **EarlyStopping**. 
- **monitor** - Métrica a monitorizar Ej: "val_loss", "val_accuracy"
- **min_delta** - Cambio mínimo en la métrica monitorizada para ser considerado como una mejora.
- **patience** - El número de épocas sin mejora después de que el entrenamiento se detendrá
- **verbose** - (0 o 1) 1 representa true, que muestra mensajes cuando la devolución de llamada realiza una acción.
- **mode** -( "auto", "min", "max") En modo "min", el entrenamiento se detendrá cuando la métrica monitorizada haya dejado de disminuir. En el modo "max", el entrenamiento se detendrá cuando la métrica supervisada haya dejado de aumentar. En el modo "auto", la dirección se deduce automáticamente del nombre de la métrica.
- **baseline** - Valor base para la métrica monitorizada. El entrenamiento se detendrá si la métrica monitorizada no muestra una mejora sobre el valor base. Por ejemplo: si la métrica monitorizada es val_accuracy, patience=10, y baseline = 50, el entrenamiento se detendrá si val_accuracy no es superior a 50 en las 10 primeras épocas.
- **restore_best_weights** - Si se restauran los pesos del modelo de la época con el mejor valor de la métrica monitorizada. Si es False, se utilizan los pesos del modelo obtenidos en el último paso del entrenamiento.


Por tanto, usando **restore_best_weights** es posible guardar los pesos del modelo de la época con el mejor rendimiento y usarlo en ella.

Un último ejemplo de uso del callback EarlyStopping sería el siguiente:

```python
early_stopping = EarlyStopping(
                              patience=10,
                              min_delta=0.001,                               
                              monitor="val_loss",
                              restore_best_weights=True
                              )
model_history = model.fit(X_train, y_train, batch_size=64, epochs = 100, validation_data = (X_test,y_test), steps_per_epoch= X_train.shape[0] // batch_size, callbacks=[early_stopping])
```

En este último ejemplo, se ha utilizado datos de entrenamiento y datos de validación. El callback EarlyStopping detendrá el entrenamiento si no hay disminución en la métrica "val_loss"(**monitor**) de nuestro modelo por lo menos hasta un valor de 0,001(**min_delta**) después o después de 10 épocas consecutivas(**patience**).